---

# CSCI 3202, Fall 2023
# Mancala Project - Outline

### Name: Victoria Bockman

In [3]:
import copy, math, time, random, pickle
from random import seed
from random import randint
from statistics import mean 
from copy import *


seed(109)

1. Play 100 games of random player against random player
    - What percentage of games does each player (1st or 2nd) win?
    - On average, how many moves does it take to win?
2. Build an AI player that uses minimax to choose the best move with a variable number of plies and a utility function we describe
    - What percentage of games does each player (AI or random) win?
    - On average, how many moves does it take to win?
3. Play 100 games with the random player against the minimax AI player at a depth of 5 plies
    - What percentage of games does each player (AI or random) win?
    - On average, how many moves does it take to win?
    - Is your AI player better than random chance? Write a paragraph or two describing why or why not.
4. Play 100 games with the random player against the Alpha-Beta AI player at a depth of 5 plies
    - How long does it take for a single game to run to completion?
    - What percentage of games does each player (AI or random) win?
    - On average, how many moves does it take to win?
    - Are your results for this part different from those for your minimax AI player? Write a paragraph or two describing why or why not.
5. (Extra Credit, 10 points). Play 100 games with the random player against the
    - Alpha-Beta AI player at a depth of 10 plies
    - How long does it take for a single game to run to completion?
    - What percentage of games does each player (AI or random) win?
    - On average, how many moves does it take to win?
    - Does increasing the number of plies improve the play for our AI player? Why or why not?

explanation for deepcopy - copy library
https://www.scaler.com/topics/copy-in-python/

In [99]:
class Mancala:
    def __init__(self, pits_per_player=6, stones_per_pit = 4):
        self.pits_per_player = pits_per_player
        self.board = [stones_per_pit] * ((pits_per_player + 1) * 2)
        self.players = 2
        self.current_player = 1
        self.moves = []
        self.p1_pits_index = [0, self.pits_per_player-1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player + 1, len(self.board) - 2]
        self.p2_mancala_index = len(self.board) - 1
    
        self.num_plays = 0
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0
        self.p1_wins = 0
        self.p2_wins = 0
        self.ties = 0
        

    def display_board(self):
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1] + 1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1] + 1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i + 1, player_1_pits[i], player_2_pits[-(i + 1)],
                                                       self.pits_per_player - i))
            else:
                print('{} -> | {} | {} | <- {}'.format(i + 1, player_1_pits[i], player_2_pits[-(i + 1)],
                                                      self.pits_per_player - i))

        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.current_player == 1 else 'P2'
        print('Turn: ' + turn)
    
    def random_move_generator(self):
                
        # Pulled from HW 5.1
        
        """
        Function to generate random valid moves with non-empty pits for the random player
        """
        #seed(109)
        
        good_move = False

        while good_move == False:
            if self.current_player == 1:
                pit = randint(0, self.pits_per_player-1)
                if self.board[pit] == 0:
                    good_move = False
                else:
                    good_move = True
                    pit = pit + 1
            else: #current_player == 2
                pit = randint(self.pits_per_player + 1, len(self.board) - 2)
                if self.board[pit] == 0:
                    good_move = False
                else:
                    good_move = True
                    pit = pit - self.pits_per_player
        return pit
        
    def generate_board(self):
        self.board = [self.stones_per_pit] * ((self.pits_per_player+1) * 2)
        self.current_player = 1
        self.moves = []
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0

    def valid_move(self, pit):
                
        # Pulled from HW 5.1
        
        # check for valid pit range

        if pit > self.pits_per_player or pit <= 0:
            return False
        
        # check if empty
        
        if self.current_player == 1:
            if self.board[pit - 1] > 0:
                return True
            else:
                return False
            
        else: #self.current_player == 2
            if self.board[pit + self.pits_per_player] > 0:
                return True
            else:
                return False

    def play(self, pit, cur_player = 0, debug = False): #set cur_player to 0 because I wasnt using it
                
        # Pulled from HW 5.1
        
        
    #    winning, winner = self.winning_eval()
    #    if winning == True:
    #        print("GAME OVER")
    #        return winner

        valid = self.valid_move(pit)
        if valid == False:
           # print("INVALID MOVE")
            return self
        else:
            self.moves.append((self.current_player, pit))
            
        # okay to play game
        
        player, pit = self.moves[-1]
        
    # we dont need to see this anymore:
       # print(f"Player {player} chose pit: {pit}")
       # print()
        
        if self.current_player == 1:
            pieces_in_pit = self.board[pit - 1]
            self.board[pit - 1] = 0
            pit_number = pit - 1
            
        else: #self.current_player == 2
            pieces_in_pit = self.board[(int((len(self.board) / 2)) - 1) + pit]
            self.board[(int((len(self.board) / 2)) - 1) + pit] = 0
            pit_number = ((int((len(self.board) / 2)) - 1) + pit)
        
        pit_number += 1
        
        empty = False
        
        
        while pieces_in_pit != 0:
            if pit_number == len(self.board):
                pit_number = 0
                
            self.board[pit_number] = self.board[pit_number] + 1
            
            if pieces_in_pit == 1:
                if self.board[pit_number] == 1:
                    empty = True
                    
            pieces_in_pit = pieces_in_pit - 1
            pit_number += 1
            
        if empty == True:
            # here, everything is distributed
            # now we check for pit across, if empty is true
            
            pit_number -= 1
            
            if self.current_player == 1:
                if pit_number == self.p1_mancala_index or pit_number == self.p2_mancala_index:
                    # do nothing, mancala
                    placeholder = 1
                elif pit_number in self.p2_pits_index:
                    # do nothing, empty pit was on other side of board
                    placeholder = 1
                else:
                    if self.board[len(self.board) - pit_number - 2] != 0:
                        #print("STEAL")
                        steal = self.board[len(self.board) - pit_number - 2]
                        self.board[len(self.board) - pit_number - 2] = 0
                        self.board[pit_number] = 0
                        self.board[self.p1_mancala_index] = self.board[self.p1_mancala_index] + 1 + steal
                    
            else: # current_player == 2
                if pit_number == self.p2_mancala_index or pit_number == self.p1_mancala_index:
                    # do nothing
                    placeholder = 1
                elif pit_number in self.p1_pits_index:
                    # do nothing, empty pit was on other side of board
                    placeholder = 1
                else:
                    if self.board[len(self.board) - pit_number - 2] != 0:
                        #print("STEAL")
                        steal = self.board[len(self.board) - pit_number - 2]
                        self.board[len(self.board) - pit_number - 2] = 0
                        self.board[pit_number] = 0
                        self.board[self.p2_mancala_index] = self.board[self.p2_mancala_index] + 1 + steal

        
        if self.current_player == 1:
            self.current_player = 2
        else: # current_player == 2
            self.current_player = 1
                
        return self
        
    def winning_eval(self):
        
        # Pulled from HW 5.1
        
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        
        player_1_total = 0
        for i in player_1_pits:
            player_1_total = player_1_total + i
            
        player_2_total = 0
        for i in player_2_pits:
            player_2_total = player_2_total + i
            
        if player_1_total == 0 or player_2_total == 0:
            if self.board[self.p1_mancala_index] > self.board[self.p2_mancala_index]:
                return True, 1
            elif self.board[self.p2_mancala_index] > self.board[self.p1_mancala_index]:
                return True, 2
            else: # its a tie
                return True, 3
        else:
            return False, 0
        
        # add utility func to end and return ?
    
    def match_analysis(self, games = 1):
        
        for x in range(games):
            move = self.random_move_generator()
            self.play(move)
            winning, winner = self.winning_eval()
            while winning == False:
                #game.display_board()
                move = self.random_move_generator()
                self.play(move)
                winning, winner = self.winning_eval()

            # winner is decided
            if winner == 1:
                return 1
            elif winner == 2:
                return 2
            else: # winner == 3
                return 3
        #return p1_wins, p2_wins, tie
        
    ### ADDING FUNCTIONS FOR THE AI PLAYER
    
    def is_terminal(self):
        # Using existing winning_eval method
        
        # 
        winning, winner = self.winning_eval()
        return winning

    def possible_valid_moves(self):
        # Return a list of possible valid moves (non-empty pits)
        
        # self.p1_pits_index = [0, self.pits_per_player-1]
        # self.p2_pits_index = [self.pits_per_player + 1, len(self.board) - 2]
        
        lst = list()
        
        if self.current_player == 1:
            for x in range(0, self.pits_per_player):
                if self.board[x] != 0: # checks 0-5
                    lst.append(x + 1) # make sure you use the appropriate pit number to be used in play()!
                else:
                    # do nothing
                    placeholder = 0
        else: #self.current_player == 2
            for x in range(0, self.pits_per_player):
                if self.board[x + self.pits_per_player + 1] != 0: # checks 7-12
                    lst.append(x + 1) # make sure you use the appropriate pit number to be used in play()!
                else:
                    # do nothing
                    placeholder = 0
        #print(lst)
        return lst
    
    def utility_function(self):
        #returns current state
        
        return self.board[self.p1_mancala_index] - self.board[self.p2_mancala_index]

In [116]:
class MancalaAI:
    def __init__(self, depth, state):
        self.depth = depth
        self.state = state
    
    def minimax(self, board, depth, maximizing_player, cur_player): 
        
        best_move = -1
        
        if board.is_terminal() or depth <= 0:
            return self.evaluate_state(board), best_move
        
        
        
        if maximizing_player: #maximize mancala for P1
            # generate all possible states for the maximizing player, and recurse 
            # until you reach the stop condition - terminal state
            
            # starting maximum evaluation
            bestValue = float('-inf')
            
            moves = board.possible_valid_moves()
            for move in moves:
                
                # copy board
                #next_board = deepcopy(board)
                next_board = pickle.loads(pickle.dumps(board))
                next_board = next_board.play(move)
                eval_state, _ = self.minimax(next_board, depth-1, False, 3 - cur_player) # minimizing players' move
                if eval_state > bestValue:
                    best_move = move
                    bestValue = eval_state
        
        else: # minimizing_player, minimize mancala for P2
            # generate all possible states for the maximizing player, and recurse
            
            # starting minimum evaluation
            bestValue = float('inf')
            
            moves = board.possible_valid_moves()
            for move in moves:
                
                # copy board
                #next_board = deepcopy(board)
                next_board = pickle.loads(pickle.dumps(board))
                next_board = next_board.play(move)
                eval_state, _ = self.minimax(next_board, depth-1, True, 3 - cur_player) # maximizing players' move
                if eval_state < bestValue:
                    best_move = move
                    bestValue = eval_state
                    
        return bestValue, best_move
    
    def minimax_alpha_beta(self, board, depth, alpha, beta, maximizing_player, cur_player):
        best_move = -1
        
        if board.is_terminal() or depth <= 0:
            return self.evaluate_state(board), best_move

        if maximizing_player:
            bestValue = float('-inf')

            moves = board.possible_valid_moves()
            for move in moves:
                next_board = pickle.loads(pickle.dumps(board))
                next_board = next_board.play(move)

                eval_state, _ = self.minimax_alpha_beta(next_board, depth - 1, alpha, beta, False, 3 - cur_player)

                if eval_state > bestValue:
                    best_move = move
                    bestValue = eval_state

                if eval_state >= beta:
                    break # Beta cut-off
                    #return bestValue, best_move
                    
                
                alpha = max(alpha, eval_state)

        else:
            bestValue = float('inf')

            moves = board.possible_valid_moves()
            for move in moves:
                next_board = pickle.loads(pickle.dumps(board))
                next_board = next_board.play(move)

                eval_state, _ = self.minimax_alpha_beta(next_board, depth - 1, alpha, beta, True, 3 - cur_player)

                if eval_state < bestValue:
                    best_move = move
                    bestValue = eval_state
                    
                if eval_state <= alpha:
                    break # Alpha cut-off
                    #return bestValue, best_move
                    
                beta = min(beta, eval_state)

                    
        return bestValue, best_move

    def evaluate_state(self, state):
        # Utility function  :- Difference between P1 mancala and p2 mancala
        
        return state.utility_function()
    
        # I had self.state here, calculating utility of original board each time?
    
    def match_random_vs_ai(self):
            # AI player is player 1
            winning, winner = self.state.winning_eval()
            num_moves = 0
            while winning == False:
                if self.state.current_player == 1:
                    copy_state = pickle.loads(pickle.dumps(self.state))
                    
                    _, move = self.minimax(copy_state, self.depth, True, copy_state.current_player)
                    self.state.play(move)
                    num_moves = num_moves + 1
    
                else: #random player
                    move = self.state.random_move_generator()
                    
                    self.state.play(move)
                    num_moves = num_moves + 1

                winning, winner = self.state.winning_eval()

            # winner is decided
            if winner == 1:
                return 1, num_moves
            elif winner == 2:
                return 2, num_moves
            else: # winner == 3
                return 3, num_moves
    def match_random_vs_alpha_beta(self):
            # AI player is player 1
            winning, winner = self.state.winning_eval()
            num_moves = 0
            while winning == False:
                if self.state.current_player == 1:
                    copy_state = pickle.loads(pickle.dumps(self.state))
                    
                    _, move = self.minimax_alpha_beta(copy_state, self.depth, float('-inf'), float('inf'), True, copy_state.current_player)
                    self.state.play(move)
                    num_moves = num_moves + 1
    
                else: #random player
                    move = self.state.random_move_generator()
                    
                    self.state.play(move)
                    num_moves = num_moves + 1

                winning, winner = self.state.winning_eval()

            # winner is decided
            if winner == 1:
                return 1, num_moves
            elif winner == 2:
                return 2, num_moves
            else: # winner == 3
                return 3, num_moves

### PART 1: Play 100 games of random player against random player

What percentage of games does each player (1st or 2nd) win?
##### On average, each player wins 50% of the time.

On average, how many moves does it take to win?

##### Average length of valid game moves: 37
###### Check code below:

In [117]:
p1_wins = 0
p2_wins = 0
tie = 0
length = []
for x in range(100):
    game = Mancala(6, 4)
    winner = game.match_analysis()
    if winner == 1:
        p1_wins = p1_wins + 1
    elif winner == 2:
        p2_wins = p2_wins + 1
    else: # winner == 3
        tie = tie + 1
    length.append(len(game.moves))
print("Player 1 wins: ", p1_wins, "%")
print("Player 2 wins: ", p2_wins, "%")
print("Tie: ", tie, "%")
print("Average game moves:", mean(length))

Player 1 wins:  53 %
Player 2 wins:  45 %
Tie:  2 %
Average game moves: 38.2


### Intermediate Results

##### So far I have only gotten through the first part of the project. It turns out I had a few bugs in my code from Homework 5.1 that I was only able to notice when I started analyzing the games in full. I have since fixed all the bugs and the random player games should be working perfectly. I just need to finish the Minimax and Alpha-Beta AI player implementation and test them against my random player for the next few parts.

### PART 2: Build an AI player that uses minimax to choose the best move with a variable number of plies and a utility function we describe

##### I'm using 1 ply here. At a depth of 2 my AI player is winning almost 100% of the time.

What percentage of games does each player (AI or random) win?
##### On average, the AI player wins 85% of the time and the Random player wins 15% of the time.

On average, how many moves does it take to win?

##### Average length of valid game moves: 25

###### Check code below:

In [102]:
p1_wins = 0
p2_wins = 0
tie = 0
length = []
for x in range(100):
    mancala_game = Mancala(6, 4)
    mancala_ai = MancalaAI(depth=1, state=mancala_game)
    winner, moves = mancala_ai.match_random_vs_ai()
    if winner == 1:
        p1_wins = p1_wins + 1
    elif winner == 2:
        p2_wins = p2_wins + 1
    else: # winner == 3
        tie = tie + 1
    length.append(moves)
print("Player 1 wins: ", p1_wins, "%")
print("Player 2 wins: ", p2_wins, "%")
print("Tie: ", tie, "%")
print("Average game moves:", mean(length))

Player 1 wins:  87 %
Player 2 wins:  13 %
Tie:  0 %
Average game moves: 25.63


### PART 3: Play 100 games with the random player against the minimax AI player at a depth of 5 plies

What percentage of games does each player (AI or random) win?
##### My AI player is winning 100% of the time.

On average, how many moves does it take to win?

##### Average length of valid game moves: 26

Is your AI player better than random chance? Write a paragraph or two describing why or why not.

##### My AI player is better than random chance because it makes decisions based on a deeper analysis of the game state. There are a few reasons why this is helpful. First, minimax is adaptable to different game situations and can adjust its strategy based on the current state of the game. Also, it is consistent in its choices based on a given game state and always plays the optimal move. Random move player have none of these characteristics, therefore it will typically lose against an agent with more knowledge.

###### Check code below:

In [109]:
p1_wins = 0
p2_wins = 0
tie = 0
length = []
for x in range(100):
    mancala_game = Mancala(6, 4)
    mancala_ai = MancalaAI(depth=5, state=mancala_game)
    winner, moves = mancala_ai.match_random_vs_ai()
    if winner == 1:
        p1_wins = p1_wins + 1
    elif winner == 2:
        p2_wins = p2_wins + 1
    else: # winner == 3
        tie = tie + 1
    length.append(moves)
print("Player 1 wins: ", p1_wins, "%")
print("Player 2 wins: ", p2_wins, "%")
print("Tie: ", tie, "%")
print("Average game moves:", mean(length))



Player 1 wins:  100 %
Player 2 wins:  0 %
Tie:  0 %
Average game moves: 27.13


### PART 4: Play 100 games with the random player against the Alpha-Beta AI player at a depth of 5 plies

How long does it take for a single game to run to completion?

##### A single game takes less than a second to run

What percentage of games does each player (AI or random) win?
##### My AI player is winning 100% of the time.

On average, how many moves does it take to win?
##### Average length of valid game moves: 27

Are your results for this part different from those for your minimax AI player? Write a paragraph or two describing why or why not.

##### My results are not different from my minimax player. This is because alpha beta is only an optimization technique to reduce the number of nodes you are visiting in the search tree. Technically, the alpha beta algorithm is almost the same as the minimax function. The algorithm prunes branches of the game tree that cannot affect the final decision, which reduces the search space.
###### Check code below for one game and 100 games:

In [123]:
p1_wins = 0
p2_wins = 0
tie = 0
length = []
for x in range(1):
    mancala_game = Mancala(6, 4)
    mancala_ai = MancalaAI(depth=5, state=mancala_game)
    winner, moves = mancala_ai.match_random_vs_alpha_beta()
    if winner == 1:
        p1_wins = p1_wins + 1
    elif winner == 2:
        p2_wins = p2_wins + 1
    else: # winner == 3
        tie = tie + 1
    length.append(moves)
print("Player 1 wins: ", p1_wins)
print("Player 2 wins: ", p2_wins)
print("Tie: ", tie)
print("Average game moves:", mean(length))

Player 1 wins:  1
Player 2 wins:  0
Tie:  0
Average game moves: 21


In [119]:
p1_wins = 0
p2_wins = 0
tie = 0
length = []
for x in range(100):
    mancala_game = Mancala(6, 4)
    mancala_ai = MancalaAI(depth=5, state=mancala_game)
    winner, moves = mancala_ai.match_random_vs_alpha_beta()
    if winner == 1:
        p1_wins = p1_wins + 1
    elif winner == 2:
        p2_wins = p2_wins + 1
    else: # winner == 3
        tie = tie + 1
    length.append(moves)
print("Player 1 wins: ", p1_wins, "%")
print("Player 2 wins: ", p2_wins, "%")
print("Tie: ", tie, "%")
print("Average game moves:", mean(length))

Player 1 wins:  100 %
Player 2 wins:  0 %
Tie:  0 %
Average game moves: 27.06


### Part 5: (Extra Credit, 10 points) Play 100 games with the random player against the Alpha-Beta AI player at a depth of 10 plies

How long does it take for a single game to run to completion?

##### A single game takes 14 seconds to run

What percentage of games does each player (AI or random) win?
##### My AI player is winning 100% of the time.

On average, how many moves does it take to win?
##### Average length of valid game moves: 25

Does increasing the number of plies improve the play for our AI player? Why or why not?
##### For a game like Mancala, I don't think increasing the number of piles to 10 is necessary for improving the gameplay of the AI player. Increasing the piles just means the function will take that much longer to run, when we already have a 100% success rate at 5 piles. In another game that is more complex, we may be able to see a difference in gameplay. As it stands, the increase to 10 piles did slightly reduce the number of game moves.

###### Check code below for one game and 100 games:

In [125]:
p1_wins = 0
p2_wins = 0
tie = 0
length = []
for x in range(1):
    mancala_game = Mancala(6, 4)
    mancala_ai = MancalaAI(depth=10, state=mancala_game)
    winner, moves = mancala_ai.match_random_vs_alpha_beta()
    if winner == 1:
        p1_wins = p1_wins + 1
    elif winner == 2:
        p2_wins = p2_wins + 1
    else: # winner == 3
        tie = tie + 1
    length.append(moves)
print("Player 1 wins: ", p1_wins)
print("Player 2 wins: ", p2_wins)
print("Tie: ", tie)
print("Average game moves:", mean(length))

Player 1 wins:  1
Player 2 wins:  0
Tie:  0
Average game moves: 18


In [126]:
p1_wins = 0
p2_wins = 0
tie = 0
length = []
for x in range(100):
    mancala_game = Mancala(6, 4)
    mancala_ai = MancalaAI(depth=10, state=mancala_game)
    winner, moves = mancala_ai.match_random_vs_alpha_beta()
    if winner == 1:
        p1_wins = p1_wins + 1
    elif winner == 2:
        p2_wins = p2_wins + 1
    else: # winner == 3
        tie = tie + 1
    length.append(moves)
print("Player 1 wins: ", p1_wins)
print("Player 2 wins: ", p2_wins)
print("Tie: ", tie)
print("Average game moves:", mean(length))

Player 1 wins:  100
Player 2 wins:  0
Tie:  0
Average game moves: 25.44
